In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import pickle

In [ ]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Insert-Generic-Name-Here/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

In [ ]:
PLT_FIG_WIDTH = 3.787
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

# Reading Data (Testing on Fisheries)

In [ ]:
ports = pd.read_pickle('./data/pkl/ports_raw.pkl')
fisheries = pd.read_csv('data/csv/fisheries_dynamic_real_points_v2.csv')

fisheries = gspp.gdf_from_df(fisheries, crs={'init':'epsg:4326'})
ports = gspp.create_port_bounds(ports, epsg=2154, port_radius=2000)

In [ ]:
gsplt.map_plot(fisheries, color=['steelblue'], title='Testing Trajectories', fontsize=10)

# [TESTING] Calculating AIS Activity 

In [ ]:
fisheries.drop(['index'], axis=1, inplace=True)

In [ ]:
fisheries.head()

* ## Outside Port

In [ ]:
ais_activity_outside_port = fisheries.groupby(['mmsi', pd.to_datetime(fisheries.ts, unit='s').dt.date, 'trip_id'], group_keys=False)\
                                        .apply(lambda df: df.ts.diff().sum()/3600).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'date', 'trip_id', '#hrs']

ais_activity_outside_port = ais_activity_outside_port.groupby(['mmsi', 'date'], group_keys=False).apply(lambda df: np.around(df['#hrs'].sum(), 3)).to_frame().reset_index()
ais_activity_outside_port.columns = ['mmsi', 'date', '#hrs']

ais_activity_outside_port

* ## Within Port

In [ ]:
ais_activity_within_port = pd.DataFrame([], columns=['mmsi', '#hrs'])

for (vesselId), df in fisheries.groupby(['mmsi']):
    breaking_points = df.loc[df.traj_id.diff() == 1].index.tolist()
    
    if (len(breaking_points) == 0):
        ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[vesselId, 0]], columns=['mmsi', '#hrs']))
        continue
    
    hrs = np.around(np.sum([df.loc[bp-1:bp,:].ts.diff().values[1] for bp in breaking_points])/3600, 3)
    ais_activity_within_port = ais_activity_within_port.append(pd.DataFrame([[vesselId, hrs]], columns=['mmsi', '#hrs']))
    
ais_activity_within_port.reset_index(drop=True, inplace=True)
ais_activity_within_port

* ## Temporal Gap

In [ ]:
ais_activity_temporal_gap = pd.DataFrame([], columns=['mmsi', 'traj_id', '#hrs'])

for (vesselId, trajId), df in fisheries.groupby(['mmsi', 'traj_id']):
    breaking_points = df.loc[df.trip_id.diff() == 1].index.tolist()
    
    if (len(breaking_points) == 0):
        ais_activity_temporal_gap = ais_activity_temporal_gap.append(pd.DataFrame([[vesselId, trajId, 0]], columns=['mmsi', 'traj_id', '#hrs']))
        continue
    
    hrs = np.around(np.sum([df.loc[bp-1:bp,:].ts.diff().sum() for bp in breaking_points])/3600, 3)
    ais_activity_temporal_gap = ais_activity_temporal_gap.append(pd.DataFrame([[vesselId, trajId, hrs]], columns=['mmsi', 'traj_id', '#hrs']))

ais_activity_temporal_gap.reset_index(drop=True, inplace=True)
ais_activity_temporal_gap = ais_activity_temporal_gap.groupby(['mmsi'], group_keys=False).apply(lambda df: np.around(df['#hrs'].sum(), 3)).to_frame().reset_index()
ais_activity_temporal_gap.columns = ['mmsi', '#hrs']
ais_activity_temporal_gap

# Reading and Joining CSVs

In [ ]:
data_folder = './data/csv/stats/nari_dynamic_ais_activity_report_V2/'

nari_dynamic_ais_activity_report_outside_port = []
nari_dynamic_ais_activity_report_within_port = []
nari_dynamic_ais_activity_report_temporal_gaps = []

for cnt, file in enumerate(os.listdir(data_folder)):
    tmp = pd.read_csv(os.path.join(data_folder, file))
    if 'outside' in file: 
        print (f'\'outside\' in: {file}')
        nari_dynamic_ais_activity_report_outside_port.append(tmp)
    elif 'within' in file: 
        print (f'\'within\' in: {file}')
        nari_dynamic_ais_activity_report_within_port.append(tmp)
    elif 'temporal' in file: 
        print (f'\'temporal\' in: {file}')
        nari_dynamic_ais_activity_report_temporal_gaps.append(tmp)
        
nari_dynamic_ais_activity_report_outside_port = pd.concat(nari_dynamic_ais_activity_report_outside_port, ignore_index=True)
nari_dynamic_ais_activity_report_within_port = pd.concat(nari_dynamic_ais_activity_report_within_port, ignore_index=True)
nari_dynamic_ais_activity_report_temporal_gaps = pd.concat(nari_dynamic_ais_activity_report_temporal_gaps, ignore_index=True)

In [ ]:
nari_dynamic_ais_activity_report_outside_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_outside_port_v2.csv', index=False, header=True)
nari_dynamic_ais_activity_report_within_port.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_within_port_v2.csv', index=False, header=True)
nari_dynamic_ais_activity_report_temporal_gaps.to_csv('./data/csv/stats/nari_dynamic_ais_activity_report_temporal_gaps_v2.csv', index=False, header=True)

# Making the Pie Chart

In [ ]:
df = pd.DataFrame({'#Hours': [nari_dynamic_ais_activity_report_outside_port['#hrs'].sum(), nari_dynamic_ais_activity_report_within_port['#hrs'].sum(), nari_dynamic_ais_activity_report_temporal_gaps['#hrs'].sum()]},
                 index=['Outside Port', 'Within Port', 'Unknown '])
print (df)
explode = (0.2, 0, 0)  
ax = df.plot.pie(y='#Hours', 
                   figsize=(PLT_FIG_WIDTH, PLT_FIG_HEIGHT), 
                   explode=explode,
                   startangle=0,
                   autopct='%1.1f%%', 
                   pctdistance=0.80,
                   cmap='tab20c',
                   legend=False)
ax.axis('equal')

for text in ax.texts:
    text.set_fontsize(8)
    if ' ' in text.get_text():
        text.set_color('grey')

plt.ylabel('')
plt.savefig('AIS_Activity_Pie_Chart_V2.pdf', dpi=300)